In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r /content/drive/'My Drive'/'Colab Notebooks'/'9d34462453e311ea.zip' /content/ 

In [3]:
ls

9d34462453e311ea.zip  drive/  sample_data/


In [4]:
!unzip 9d34462453e311ea.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/Train Images/image5794.jpg  
  inflating: dataset/Train Images/image6162.jpg  
  inflating: dataset/Train Images/image3291.jpg  
  inflating: dataset/Train Images/image4189.jpg  
  inflating: dataset/Train Images/image9440.jpg  
  inflating: dataset/Train Images/image1160.jpg  
  inflating: dataset/Train Images/image225.jpg  
  inflating: dataset/Train Images/image92.jpg  
  inflating: dataset/Train Images/image7016.jpg  
  inflating: dataset/Train Images/image1891.jpg  
  inflating: dataset/Train Images/image2099.jpg  
  inflating: dataset/Train Images/image10509.jpg  
  inflating: dataset/Train Images/image10371.jpg  
  inflating: dataset/Train Images/image5258.jpg  
  inflating: dataset/Train Images/image10090.jpg  
  inflating: dataset/Train Images/image2296.jpg  
  inflating: dataset/Train Images/image10361.jpg  
  inflating: dataset/Train Images/image8111.jpg  
  inflating: dataset/Train Images/image10696.jpg

In [0]:
import os
import numpy as np
import pandas as pd
import cv2

In [6]:
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')

train_df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [7]:
class_map = {
    'Food': 0,
    'Attire': 1,
    'Decorationandsignage': 2,
    'misc': 3
}

inverse_class_map = {
    0: 'Food',
    1: 'Attire',
    2: 'Decorationandsignage',
    3: 'misc'
}

train_df['Class'] = train_df['Class'].map(class_map).astype(np.uint8)
train_df.head()

,Image,Class
0,image7042.jpg,0
1,image3327.jpg,3
2,image10335.jpg,1
3,image8019.jpg,0
4,image2128.jpg,1


In [0]:
train_path = 'dataset/Train Images/'
test_path = 'dataset/Test Images/'

h, w = 150, 150

train_images, train_labels = [], []

for i in range(len(train_df.Image)):
  train_image = cv2.imread(train_path + str(train_df.Image[i]))
  train_image = cv2.resize(train_image, (h, w))
  train_image = train_image.astype('float32')
  train_images.append(train_image)
  train_labels.append(train_df.Class[i])

test_images = []

for i in range(len(test_df.Image)):
  test_image = cv2.imread(test_path + str(test_df.Image[i]))
  test_image = cv2.resize(test_image, (h, w))
  test_image = test_image.astype('float32')
  test_images.append(test_image)

train_images = np.array(train_images)
test_images = np.array(test_images)

In [9]:
# !pip install tensorflow-gpu==2.0

import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [0]:
# !pip show tensorflow
# import keras

In [10]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.python.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_images, to_categorical(train_labels), test_size=0.2, random_state=42)

In [0]:
# ResNet50

base_model_res = ResNet50(
    weights='imagenet',
    include_top=False, 
    input_shape=(150, 150, 3), 
    pooling='avg'
)

In [0]:
base_model_res.trainable = False

model_res = Sequential([
  base_model_res,
  Dropout(0.2),
  Dense(4, activation='softmax')
])

model_res.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_res.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 8196      
Total params: 23,595,908
Trainable params: 8,196
Non-trainable params: 23,587,712
_________________________________________________________________


In [0]:
batch_size = 32
epochs = 50

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

model_res.fit(datagen.flow(X_train, y_train, batch_size=batch_size), validation_data= (X_test, y_test),
                    steps_per_epoch=len(X_train) / batch_size, epochs=epochs)

Epoch 1/50
150/149 [==============================] - 32s 217ms/step - loss: 1.3320 - accuracy: 0.3588 - val_loss: 4.1126 - val_accuracy: 0.2715
Epoch 2/50
150/149 [==============================] - 30s 203ms/step - loss: 1.3168 - accuracy: 0.3663 - val_loss: 6.1552 - val_accuracy: 0.2765
Epoch 3/50
150/149 [==============================] - 30s 202ms/step - loss: 1.3029 - accuracy: 0.3817 - val_loss: 9.0957 - val_accuracy: 0.2749
Epoch 4/50
150/149 [==============================] - 30s 203ms/step - loss: 1.2970 - accuracy: 0.3855 - val_loss: 12.4392 - val_accuracy: 0.2757
Epoch 5/50
150/149 [==============================] - 30s 200ms/step - loss: 1.2904 - accuracy: 0.3968 - val_loss: 14.8981 - val_accuracy: 0.2757
Epoch 6/50
150/149 [==============================] - 30s 200ms/step - loss: 1.2842 - accuracy: 0.3991 - val_loss: 17.3772 - val_accuracy: 0.2757
Epoch 7/50
150/149 [==============================] - 30s 201ms/step - loss: 1.2794 - accuracy: 0.4070 - val_loss: 19.0994 - va

KeyboardInterrupt: ignored

In [12]:
# VGG16

base_model_vgg = VGG16(weights='imagenet',include_top=False, input_shape=(150, 150, 3), pooling='avg')
base_model_vgg.trainable = False

# model = Model(inputs=base_model.input, outputs=)
# model_vgg = Sequential(Dropout(0.2))
model_vgg = Sequential([
  base_model_vgg,
  Dropout(0.2),
  Dense(4, activation='softmax')
])

# model_vgg.add(base_model_vgg)

model_vgg.add(Dense(256,activation='relu'))
model_vgg.add(Dense(4,activation='softmax'))

model_vgg.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_vgg.summary()
# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

58892288/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 2052      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1280      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1028      
Total params: 14,719,048
Trainable params: 4,360
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
batch_size = 32
epochs = 20

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]

model_vgg.fit(datagen.flow(X_train, y_train, batch_size=batch_size), validation_data= (X_test, y_test), steps_per_epoch=len(X_train) / batch_size, epochs=epochs)

Epoch 1/20
150/149 [==============================] - 25s 166ms/step - loss: 1.2735 - accuracy: 0.3863 - val_loss: 1.1994 - val_accuracy: 0.4511
Epoch 2/20
150/149 [==============================] - 24s 159ms/step - loss: 1.1627 - accuracy: 0.4448 - val_loss: 1.2158 - val_accuracy: 0.4578
Epoch 3/20
150/149 [==============================] - 24s 158ms/step - loss: 1.1358 - accuracy: 0.4628 - val_loss: 1.1919 - val_accuracy: 0.4553
Epoch 4/20
150/149 [==============================] - 23s 157ms/step - loss: 1.0780 - accuracy: 0.5192 - val_loss: 1.2377 - val_accuracy: 0.4419
Epoch 5/20
150/149 [==============================] - 23s 156ms/step - loss: 1.0039 - accuracy: 0.5850 - val_loss: 1.3274 - val_accuracy: 0.4436
Epoch 6/20
150/149 [==============================] - 23s 156ms/step - loss: 0.9750 - accuracy: 0.5863 - val_loss: 1.3438 - val_accuracy: 0.4528
Epoch 7/20
150/149 [==============================] - 23s 154ms/step - loss: 0.9552 - accuracy: 0.6055 - val_loss: 1.3740 - val_ac

In [0]:
# Xception

base_model_xcep = Xception(weights='imagenet',include_top=False, input_shape=(h, w, 3), pooling='avg')
base_model_xcep.trainable = False

# model = Model(inputs=base_model.input, outputs=)
# model_xcep = Sequential(Dropout(0.2))
model_xcep = Sequential([
  base_model_xcep,
  Dropout(0.2),
  Dense(4, activation='softmax')
])

# model_xcep.add(base_model_xcep)

model_xcep.add(Dense(256,activation='relu'))
model_xcep.add(Dense(4,activation='softmax'))

model_xcep.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_xcep.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 2048)              20861480  
_________________________________________________________________
dropout_7 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 8196      
_________________________________________________________________
dense_19 (Dense)             (None, 256)               1280      
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 1028      
Total params: 20,871,984
Trainable params: 10,504
Non-trainable params: 20,861,480
_________________________________________________________________


In [0]:
batch_size = 32
epochs = 50

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range=10,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]

model_xcep.fit(datagen.flow(X_train, y_train, batch_size=batch_size), validation_data= (X_test, y_test),
                    steps_per_epoch=len(X_train) / batch_size, epochs=epochs, callbacks = callbacks)

Epoch 1/50
150/149 [==============================] - 34s 228ms/step - loss: 1.0811 - accuracy: 0.5474 - val_loss: 1.8659 - val_accuracy: 0.1947 - lr: 0.0010
Epoch 2/50
150/149 [==============================] - 34s 224ms/step - loss: 0.8697 - accuracy: 0.6703 - val_loss: 2.2264 - val_accuracy: 0.2515 - lr: 0.0010
Epoch 3/50
150/149 [==============================] - 33s 221ms/step - loss: 0.7967 - accuracy: 0.7052 - val_loss: 2.4986 - val_accuracy: 0.2556 - lr: 0.0010
Epoch 4/50
150/149 [==============================] - 33s 223ms/step - loss: 0.7550 - accuracy: 0.7269 - val_loss: 2.7159 - val_accuracy: 0.2556 - lr: 0.0010
Epoch 5/50
150/149 [==============================] - 33s 222ms/step - loss: 0.7481 - accuracy: 0.7294 - val_loss: 2.6063 - val_accuracy: 0.2682 - lr: 0.0010
Epoch 6/50
150/149 [==============================] - 33s 222ms/step - loss: 0.7377 - accuracy: 0.7292 - val_loss: 2.5156 - val_accuracy: 0.2707 - lr: 0.0010
Epoch 7/50
150/149 [==============================] 

In [0]:
# training_generator = datagen.flow(X_train, y_train, batch_size=batch_size,subset='training',seed=7)
# validation_generator = datagen.flow(X_train, y_train, batch_size=batch_size,subset='validation',seed=7)

# model.fit(
#         train_generator,
#         steps_per_epoch=200,
#         epochs= epochs,
#         validation_data= validation_generator,
#         validation_steps= 80)


# model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[EarlyStopping(min_delta=0.01)])

In [15]:
labels = model_vgg.predict(test_images)
print(labels[:4])

[[9.1392690e-01 1.9653274e-03 2.8532229e-02 5.5575546e-02]
 [5.6120980e-01 3.8419834e-01 3.3657590e-04 5.4255236e-02]
 [9.1392690e-01 1.9653274e-03 2.8532237e-02 5.5575546e-02]
 [9.1392690e-01 1.9653274e-03 2.8532237e-02 5.5575546e-02]]


In [16]:
label = [np.argmax(i) for i in labels]
print(label[:20])

[0, 0, 0, 0, 1, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 0, 1, 1, 3, 3]


In [17]:
class_label = [inverse_class_map[x] for x in label]
print(class_label[:20])

['Food', 'Food', 'Food', 'Food', 'Attire', 'Attire', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'Food', 'Attire', 'Food', 'Food', 'Attire', 'Attire', 'misc', 'misc']


In [18]:
submission = pd.DataFrame({ 'Image': test_df.Image, 'Class': class_label })
submission.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [0]:
submission.to_csv('sub.csv', index=False)

In [20]:
ls

9d34462453e311ea.zip  dataset/  drive/  sample_data/  sub.csv
